In [2]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [3]:
def remove_extra_sapces(df):
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
    return df

In [4]:
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
# Find empty rows (rows with all NaN values)

def extract_tables(df):
    start_row = 0 
    end_row = 140
    start_column_idx =0 
    end_column_idx = 14
    table_df = df.iloc[start_row:end_row , start_column_idx:end_column_idx ]

    empty_rows = table_df[table_df.isnull().all(axis=1)]

    # Initialize variables for splitting
    split_dataframes = []
    saved_dataframes = {}
    start_index = 0

    # Iterate through empty row indices and split DataFrame
    for index in empty_rows.index:
        split_dataframes.append(table_df.iloc[start_index:index])
        start_index = index + 1

    # Add the remaining portion of the DataFrame
    if start_index < len(table_df):
        split_dataframes.append(table_df.iloc[start_index:])

    # Print the split DataFrames
    hotel_df = pd.DataFrame()
    product_df = pd.DataFrame()
    for i, df in enumerate(split_dataframes):
        df.dropna(how='all', inplace=True)
        df.dropna(axis=1, how='all', inplace=True)
        df.columns = df.iloc[0]
        df = df[1:]
        new_sheet_name = df.columns[0].lower().replace(' ', '_')
        df = remove_extra_sapces(df) 
        saved_dataframes[new_sheet_name] = df

    return saved_dataframes

In [5]:
hotel_df = pd.read_excel("Kopie_van_Model_Vaya.xlsx",sheet_name="Employee model")
dataframes=extract_tables(hotel_df)

C:\Users\AmolShyamraoBhasme\AppData\Local\Temp\ipykernel_7836\370034743.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(how='all', inplace=True)
C:\Users\AmolShyamraoBhasme\AppData\Local\Temp\ipykernel_7836\370034743.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, how='all', inplace=True)
C:\Users\AmolShyamraoBhasme\AppData\Local\Temp\ipykernel_7836\370034743.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [6]:
def hotel_characteristics_table(dataframes):
    product_characteristics_df ={}
    hotel_characteristics_df ={}
    management_type = None
    clean_room_every_x_day = None
    if "characteristics" in dataframes:
        df = dataframes["characteristics"].iloc[:,[2,3]].T
        df.dropna(axis=1, how='all', inplace=True)
        new_header = df.iloc[0]  # Get the first row as the new header
        df = df[1:]  # Remove the first row (old header)
        df.columns = new_header  
        df= df.reset_index(drop=True)
        df.columns = df.columns.str.lower()
        df.columns=df.columns.str.replace(" ","_")
        for col in df.columns:     
            product_characteristics_df[col]= df[col][0]
            
    if "characteristics" in dataframes:
        df = dataframes["characteristics"].iloc[:,[0,1]].T
        df.dropna(axis=1, how='all', inplace=True)
        new_header = df.iloc[0]  # Get the first row as the new header
        df = df[1:]  # Remove the first row (old header)
        df.columns = new_header  
        df= df.reset_index(drop=True)
        df.columns = df.columns.str.lower()
        df.columns=df.columns.str.replace(" ","_")
        for col in df.columns:
            hotel_characteristics_df[col]= df[col][0]

    return product_characteristics_df, hotel_characteristics_df



In [7]:
characteristics=hotel_characteristics_table(dataframes)

In [8]:
characteristics

({'product_characteristics': 0,
  'clean_room_once_every_x_days': 0,
  'clean_common_every_x_days': 3,
  'menu_type': 'complex',
  'selfservice_breakfast': 'yes',
  'mobile_check-in/out_entry/exit': 'no',
  'mobile_ordering_/paying': 'no',
  'premade_meals_option': 'no',
  '%_guests_take_premade': 0.05,
  'management_type': 'managers work',
  'chef_type': 'chef-assistant'},
 {'hotel_characteristics': nan,
  'rooms_hotel': 100,
  'guest_capacity': 250,
  'm2_general_space': 1000,
  'seating_capacity_restaurant': 50,
  'spa_treatment_capacity': 2,
  'work_hours': nan,
  'target_average_workhours': datetime.time(8, 0)})

In [9]:
import datetime
def efficiency_labor_management(dataframes):
    labor_management_dfs= []
    labor_management_df1={}
    labor_management_df2={}
    if "effiency_of_labor_(management)" in dataframes:
        df = dataframes["effiency_of_labor_(management)"]
        df1 = df.iloc[:,[0,1]].T
        df2 = df.iloc[:,[2,3]].T
        df3 = df.iloc[:,[4,5]].T

        new_header = df1.iloc[0]  # Get the first row as the new header
        df1 = df1[1:]  # Remove the first row (old header)
        df1.columns = new_header
        df1= df1.reset_index(drop=True)
        df1.rename(columns={np.nan: 'split'}, inplace=True)
        new_header = df2.iloc[0]  # Get the first row as the new header
        df2 = df2[1:]  # Remove the first row (old header)
        df2.columns = new_header
        df2= df2.reset_index(drop=True)
        df2.rename(columns={np.nan: 'split'}, inplace=True)

        new_header = df3.iloc[0]
        df3 = df3[1:]  # Remove the first row (old header)
        df3.columns = new_header
        df3= df3.reset_index(drop=True)
        df3.rename(columns={np.nan: 'split'}, inplace=True)

        if "Totaal" in df1.columns:
            df1.rename(columns={'Totaal': 'Total'}, inplace=True)
        if "Totaal" in df2.columns:
            df2.rename(columns={'Totaal': 'Total'},inplace= True)  
        if "Totaal" in df3.columns:
            df3.rename(columns={'Totaal': 'Total'},inplace= True)  
           
        labor_management_df1= df1  
        labor_management_df2= df2
        labor_management_df3= df3

        labor_management_dfs.append(labor_management_df1)    
        labor_management_dfs.append(labor_management_df2) 
        labor_management_dfs.append(labor_management_df3) 
        saved_dataframes={}
        for data in labor_management_dfs:
            total_indices = [i for i, col in enumerate(data.columns) if 'split' in col or 'Total' in col]

            # Initialize a list to store section DataFrames
            section_dfs = []

            # Split the DataFrame based on "Total" columns
            start_idx = 0
            for total_idx in total_indices:
                section_df = data.iloc[:, start_idx:total_idx + 1]
                section_dfs.append(section_df)
                start_idx = total_idx + 1

            # Display the section DataFrames
            for i, section_df in enumerate(section_dfs):
                dattime_cols =[] 
                labor_df=pd.DataFrame()
                for column in section_df.columns:
                # Check the data type of the first value in the column
            #         print(isinstance(section_df[column][0], datetime.time))
                    if isinstance(section_df[column][0], datetime.time):
                        # Handle datetime.time objects differently
                         dattime_cols.append(column) 
            #             section_df[column] = section_df[column].apply(lambda x: pd.Timedelta(hours=x.hour, minutes=x.minute, seconds=x.second))
                    else:
                        # Convert other values to timedelta
                        pass

                labor_df=section_df[dattime_cols]
                labor_df=labor_df.applymap(lambda x: datetime.timedelta(hours=x.hour, minutes=x.minute, seconds=x.second))
                labor_df.columns = labor_df.columns.str.lower()
                labor_df.columns=labor_df.columns.str.replace(" ","_")
                if labor_df.sum(axis=1).dtype == 'float64':
                    pass
                elif "Total" in section_df.columns:
                    section_df["Total"]=labor_df.iloc[:, :-1].sum(axis=1).apply(lambda x: datetime.time(x.seconds // 3600, (x.seconds // 60) % 60, x.seconds % 60))
                    saved_dataframes[section_df.columns[0].lower().replace(' ', '_')] =section_df
                elif 'split' in section_df.columns and len(section_df.columns)>1:
                    for col in section_df.columns:
                        saved_dataframes[col.lower().replace(' ', '_')] =  section_df[col]

        
    return saved_dataframes

In [10]:
labor_management_dfs = efficiency_labor_management(dataframes)

C:\Users\AmolShyamraoBhasme\AppData\Local\Temp\ipykernel_7836\2399251704.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  section_df["Total"]=labor_df.iloc[:, :-1].sum(axis=1).apply(lambda x: datetime.time(x.seconds // 3600, (x.seconds // 60) % 60, x.seconds % 60))
C:\Users\AmolShyamraoBhasme\AppData\Local\Temp\ipykernel_7836\2399251704.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  section_df["Total"]=labor_df.iloc[:, :-1].sum(axis=1).apply(lambda x: datetime.time(x.seconds // 3600, (x.seconds //

In [11]:
from datetime import datetime
import datetime as dt
def add_date_columns(data):
    peak_demand=data

    length=len(peak_demand.columns)

    rename_col =peak_demand.columns[0]
    # Get today's date
    start_date = datetime.today().date()

    # Replace NaN column names with the new dates
    new_column_names = peak_demand.columns.tolist()
    new_column_names[1:]=  [start_date + timedelta(days=i) for i in range(1,length)]

    if len(peak_demand.columns[1:].dropna())==0:
        new_column_names[0] = peak_demand.iloc[0][0]
        peak_demand.columns = new_column_names
        peak_demand = peak_demand.drop(peak_demand.index[0])
    
    peak_demand.columns = new_column_names
    peak_demand=remove_extra_sapces(peak_demand)

    return peak_demand


In [12]:
from decimal import Decimal, ROUND_HALF_UP
def roundOf(hours):
    decimal_value = Decimal(hours)
    rounded_value = decimal_value.quantize(Decimal("0.00"), rounding=ROUND_HALF_UP)
    return rounded_value

In [13]:
def convert_to_hours(time):
    hours=time.hour + time.minute / 60 + time.second / 3600
    return hours

In [14]:
def kitchen_assistants_during_diner_cal(menue_type, culinairy_complex_total, culinairy_simple_total, boardtype2, target_kitchen_cleaning_time, dining_split_shifts, dining_1stshift_end, start_prepare_dinner, dining_2ndtshift_end, dining_2ndtshift_start,chef_type , cooking_simple_total, cooking_complex_total):
    if menue_type == "complex":
        numerator = (culinairy_complex_total.iloc[0] * boardtype2.iloc[0] + target_kitchen_cleaning_time.iloc[0])
        if dining_split_shifts.iloc[0] == "no":
            
            denominator = dining_1stshift_end - start_prepare_dinner.iloc[0]
        else:
            denominator = (dining_2ndtshift_end - dining_2ndtshift_start) + (dining_1stshift_end - start_prepare_dinner.iloc[0])
        
        if chef_type == "chef-assistant":
            inner_value = 1 - min(cooking_simple_total.iloc[0] * boardtype2.iloc[0], 1)
            
        else:
            inner_value = 1- max(cooking_complex_total.iloc[0] * boardtype2.iloc[0], 1)
       
    else:
        numerator = (culinairy_simple_total.iloc[0] * boardtype2.iloc[0] + target_kitchen_cleaning_time.iloc[0])
        if dining_split_shifts.iloc[0] == "no":
            denominator = dining_1stshift_end - start_prepare_dinner.iloc[0]
        else:
            denominator = (dining_2ndtshift_end - dining_2ndtshift_start) + (dining_1stshift_end - start_prepare_dinner.iloc[0])
        
        if chef_type  == "chef-assistant":
            inner_value = 1- min(cooking_simple_total.iloc[0] * boardtype2.iloc[0], 1)
        else:
            inner_value = 1- max(cooking_complex_total.iloc[0] * boardtype2.iloc[0], 1)    
  
    result =  numerator / denominator - inner_value

    return result

# Replace the following values with your actual data


In [15]:
def waiters_during_diner_cal(mobile_ordering_paying,waitering_normal,waitering_mobile, dining_split_shifts, table_setting_total, boardtype2, occupancy_rates, guests_take_premade, premade_meals_option, dining_1stshift_end, dining_1stshift_start, dining_2ndtshift_end, dining_2ndtshift_start):
    if mobile_ordering_paying == "no":
        numerator = ((waitering_normal.iloc[0] + (1 if dining_split_shifts.iloc[0] == "no" else 2) * table_setting_total.iloc[0]) * boardtype2.iloc[0] * occupancy_rates.iloc[0] * (1 - guests_take_premade)) if premade_meals_option== "yes" else ((waitering_normal.iloc[0] + (1 if dining_split_shifts.iloc[0] == "no" else 2) * table_setting_total.iloc[0]) * boardtype2.iloc[0] * occupancy_rates.iloc[0])
    else:
        numerator = ((waitering_mobile.iloc[0] + (1 if dining_split_shifts.iloc[0] == "no" else 2) * table_setting_total.iloc[0]) * boardtype2.iloc[0] * occupancy_rates.iloc[0] * (1 - guests_take_premade)) if premade_meals_option == "yes" else ((waitering_mobile.iloc[0] + (1 if dining_split_shifts.iloc[0] == "no" else 2) * table_setting_total.iloc[0]) * boardtype2.iloc[0] * occupancy_rates.iloc[0])

    # Calculate the denominato
    denominator = (dining_1stshift_end - dining_1stshift_start) if dining_split_shifts.iloc[0] == "no" else ((dining_2ndtshift_end - dining_2ndtshift_start) + (dining_1stshift_end - dining_1stshift_start))

    # Calculate the final result
    result = numerator / denominator
    return result


In [18]:
import datetime as dt
import numpy as np
def peak_demand(dataframes):
    labor_demand_df = add_date_columns(dataframes["demand_for_labor_(protel)"])
    peak_demand_df = add_date_columns(dataframes["peak_demand_times_for_labor_(management)"])
    meal_times_df = add_date_columns(dataframes["meal_times"])
    bar_df = add_date_columns(dataframes["bar"])
    spa_wellness_df = add_date_columns(dataframes["spa_and_wellness"])

    labor_demand_df = remove_extra_sapces(labor_demand_df)
    peak_demand_df = remove_extra_sapces(peak_demand_df)
    meal_times_df = remove_extra_sapces(meal_times_df)
    bar_df = remove_extra_sapces(bar_df)
    spa_wellness_df = remove_extra_sapces(spa_wellness_df)

    head_of_reception = pd.DataFrame()
    head_of_cleaning = pd.DataFrame()
    receptionists = pd.DataFrame()
    head_of_restaurant = pd.DataFrame()
    room_cleaners = pd.DataFrame()
    common_area_clean = pd.DataFrame()
    laundry_attends = pd.DataFrame()
    chef = pd.DataFrame()
    kitchen_assistants = pd.DataFrame()
    waiters = pd.DataFrame()
    bartenders = pd.DataFrame()
    spa_wellness = pd.DataFrame()

    length = len(dataframes["peak_demand_times_for_labor_(management)"])
    start_date = datetime.today().date()+timedelta(days=1) 

    # Create a list of new column names with dates
    new_column_names = [start_date + timedelta(days=i) for i in range(length-1)]

    head_of_reception["Head of Reception"]=["Regular"]
    receptionists["Receptionists"]=["Receptionists check-in", "Receptionists check-out", "Receptionists regular"]
    head_of_cleaning["Head of Cleaning"]=["Regular"]
    head_of_restaurant["Head of Restaurant"]=["Regular"]
    common_area_clean["Common area cleaners"]=["Regular"]

    room_cleaners["Room cleaners"]=["Room cleaner between checks","Room cleaner regular"]
    laundry_attends["Laundry attendants"]=["Laundry between checks","Regular"]
    chef["Chef"]=["Chef during diner"]
    kitchen_assistants["Kitchen assistants"]= ["Kitchen assistants during breakfast", "Kitchen assistants during diner"]
    waiters["Waiters"]= ["Waiters during breakfast","Waiters during diner"]
    bartenders["Bartenders"]=["Bartenders during bartime"]
    spa_wellness["Spa therapist"] = ["Spa therapists during massage time"]
    for i in new_column_names:
        if characteristics[0]["management_type"]!="no managers":
            regular = 1
        else:
            regular = 0
        head_of_reception[i] = [regular]
        head_of_cleaning[i] = [regular]
        head_of_restaurant[i]= [regular]

    reception_check_in_total= labor_management_dfs["reception_check-in"]["Total"].apply(convert_to_hours)
    reception_check_out_total= labor_management_dfs["reception_check-out"]["Total"].apply(convert_to_hours)
    reception_mobile_check_in_total= labor_management_dfs["reception_check-in_(mobile)"]["Total"].apply(convert_to_hours)
    reception_mobile_check_out_total= labor_management_dfs["reception_check-out_(mobile)"]["Total"].apply(convert_to_hours)
    mobile_check_in_out= characteristics[0]["mobile_check-in/out_entry/exit"]

    hourskeeping_staying_total= labor_management_dfs["housekeeping_(staying_guests)"]["Total"].apply(convert_to_hours)
    hourskeeping_new_total= labor_management_dfs["housekeeping_(new_guests)"]["Total"].apply(convert_to_hours)

    clean_room_count=characteristics[0]["clean_room_once_every_x_days"]
    clean_common_count=characteristics[0]["clean_common_every_x_days"]
    rooms_hotel_count=characteristics[1]["rooms_hotel"]

    target_average_workhours=convert_to_hours(characteristics[1]["target_average_workhours"])
    m2_gerneral_space=characteristics[1]["m2_general_space"]

    hourskeeping_common_area_total= labor_management_dfs["housekeeping_(common_areas)"]["Total"].apply(convert_to_hours)
    laundry_time = labor_management_dfs["target_laundry_transfer_time_per_room"].apply(convert_to_hours)

    laundry_storage_total = labor_management_dfs["laundry_storage"]["Total"].apply(convert_to_hours)

    chef_type =  characteristics[0]["chef_type"]
    menue_type = characteristics[0]["menu_type"]
    mobile_ordering_paying = characteristics[0]["mobile_ordering_/paying"]
    guests_take_premade = characteristics[0]["%_guests_take_premade"]
    premade_meals_option = characteristics[0]["premade_meals_option"]
    cooking_complex_total = labor_management_dfs["cooking_(complex)"]["Total"].apply(convert_to_hours)
    breakfast_target_time = labor_management_dfs["target_preperation_time"].apply(convert_to_hours)
    breakfast_target_buffet_time = labor_management_dfs["target_buffet_set-up_and_restocking"].apply(convert_to_hours)
    target_kitchen_cleaning_time = labor_management_dfs["target_kitchen_cleaning_time"].apply(convert_to_hours)

    cooking_simple_total = labor_management_dfs["cooking_(simple)"]["Total"].apply(convert_to_hours)
    culinairy_complex_total = labor_management_dfs["culinairy_operations_ex_cooking_(complex)"]["Total"].apply(convert_to_hours)
    culinairy_simple_total = labor_management_dfs["culinairy_operations_ex_cooking_(simple)"]["Total"].apply(convert_to_hours)

    table_setting_total = labor_management_dfs["table_setting"]["Total"].apply(convert_to_hours)
    waitering_normal = labor_management_dfs["waitering_(normal)"]["Total"].apply(convert_to_hours)
    waitering_mobile = labor_management_dfs["waitering_(mobile)"]["Total"].apply(convert_to_hours)


    dining_shifts_df = pd.DataFrame()
    dining_shifts_df["Dining Shifts"] = ["dining_1stshift_start", "dining_1stshift_end","dining_2ndshift_start","dining_2ndshift_end"]

    for reciption in new_column_names:
        start_check_in=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-in peak start"][reciption].apply(convert_to_hours)
        end_check_in=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-in peak end"][reciption].apply(convert_to_hours)
        guest_within_peak_end =peak_demand_df[peak_demand_df["Check-in and out times"] == "guests within peak hours"][reciption]
        new_guests= labor_demand_df[labor_demand_df["Reservations"] == "New guests"][reciption]

        start_check_out=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-out peak start"][reciption].apply(convert_to_hours)
        end_check_out=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-out peak end"][reciption].apply(convert_to_hours)
        leaving_guests= labor_demand_df[labor_demand_df["Reservations"] == "Leaving guests"][reciption]

        new_room= labor_demand_df[labor_demand_df["Reservations"] == "New rooms"][reciption]
        checkin_earliest =peak_demand_df[peak_demand_df["Check-in and out times"] == "check-in time (earliest)"][reciption].apply(convert_to_hours)
        checkout_latest =peak_demand_df[peak_demand_df["Check-in and out times"] == "check-out time (latest)"][reciption].apply(convert_to_hours)

        occupancy_rates= labor_demand_df[labor_demand_df["Reservations"] == "Occupancy rate"][reciption]
        boardtype2= labor_demand_df[labor_demand_df["Reservations"] == "Boardtype2"][reciption]
        boardtype1= labor_demand_df[labor_demand_df["Reservations"] == "Boardtype1"][reciption]
        massage_reservations= labor_demand_df[labor_demand_df["Reservations"] == "Massage reservations"][reciption]

        dining_end = meal_times_df[meal_times_df["Meal times"] == "dining end"][reciption].apply(convert_to_hours)
        start_prepare_dinner = meal_times_df[meal_times_df["Meal times"] == "start preperation diner"][reciption].apply(convert_to_hours)
        breakfast_start = meal_times_df[meal_times_df["Meal times"] == "breakfast start"][reciption].apply(convert_to_hours)
        breakfast_end = meal_times_df[meal_times_df["Meal times"] == "breakfast end"][reciption].apply(convert_to_hours)
        breakfast_prepare_start = meal_times_df[meal_times_df["Meal times"] == "breakfast preperation"][reciption].apply(convert_to_hours)

        dining_split_shifts = meal_times_df[meal_times_df["Meal times"] == "dining split shifts"][reciption]
        dining_start = meal_times_df[meal_times_df["Meal times"] == "dining start"][reciption].apply(convert_to_hours)

        bar_open = bar_df[bar_df["Bar"] == "Bar open"][reciption].apply(convert_to_hours)
        bar_close = bar_df[bar_df["Bar"] == "Bar closed"][reciption].apply(convert_to_hours)

        spa_massage_duration = spa_wellness_df[spa_wellness_df["Spa and Wellness"] == "Massage duration"][reciption].apply(convert_to_hours)
        wellness_open = spa_wellness_df[spa_wellness_df["Spa and Wellness"] == "Wellness open"][reciption].apply(convert_to_hours)
        wellness_closed = spa_wellness_df[spa_wellness_df["Spa and Wellness"] == "Wellness closed"][reciption].apply(convert_to_hours)

        time_between_splits = meal_times_df[meal_times_df["Meal times"] == "time between splits"][reciption].apply(convert_to_hours)
        if mobile_check_in_out == "no":

            receptionists_checkin =  (reception_check_in_total.iloc[0]*new_guests.iloc[0])/ ((end_check_in.iloc[0]-start_check_in.iloc[0])*guest_within_peak_end.iloc[0])
            receptionists.at[0,reciption]= roundOf(receptionists_checkin)
            receptionists_checkout = (reception_check_out_total.iloc[0]*leaving_guests.iloc[0]) / (end_check_out.iloc[0]-start_check_out.iloc[0])     
            receptionists.at[1,reciption]= roundOf(receptionists_checkout)
            receptionists.at[2,reciption]= roundOf(1.0)
        else:

            receptionists_checkin = (reception_mobile_check_in_total.iloc[0]*new_guests.iloc[0]) / ((end_check_in.iloc[0]-start_check_in.iloc[0])*guest_within_peak_end.iloc[0])
            receptionists.at[0,reciption]= roundOf(receptionists_checkin)
            receptionists_checkout = (reception_mobile_check_out_total.iloc[0]*leaving_guests.iloc[0]) / (end_check_out.iloc[0]-start_check_out.iloc[0])
            receptionists.at[1,reciption]= roundOf(receptionists_checkout)
            receptionists.at[2,reciption]= roundOf(1.0)

        houskeeping_room_clean=hourskeeping_staying_total.iloc[0]+hourskeeping_new_total.iloc[0] 
        clean_room_total=1+clean_room_count
        checkin_diff = checkin_earliest.iloc[0]-checkout_latest.iloc[0]
        room_cleaner_between=(new_room.iloc[0]*(houskeeping_room_clean)/clean_room_total)/checkin_diff
        room_cleaners.at[0,reciption]=roundOf(room_cleaner_between)

        room_cleaner_regular=((rooms_hotel_count * occupancy_rates.iloc[0]-new_room.iloc[0]) * hourskeeping_staying_total.iloc[0]/clean_room_total)/ target_average_workhours
        room_cleaners.at[1,reciption]=roundOf(room_cleaner_regular)

        common_area_clean_result=(hourskeeping_common_area_total.iloc[0]*m2_gerneral_space/(clean_common_count+1))/target_average_workhours
        common_area_clean.at[0, reciption] = roundOf(common_area_clean_result)

        laundary_between_result=(new_room.iloc[0]*(laundry_time.iloc[0])/(checkin_diff))
        laundry_attends.at[0, reciption] = roundOf(laundary_between_result)

        laundary_regular_result=(laundry_time.iloc[0]*rooms_hotel_count*occupancy_rates.iloc[0]+laundry_storage_total.iloc[0])/target_average_workhours
        laundry_attends.at[1, reciption] = roundOf(laundary_regular_result)

        chef_durring_dinner = 0
        if chef_type != "no chef":
            chef_durring_dinner = cooking_complex_total.iloc[0]*boardtype2.iloc[0]/(dining_end.iloc[0]-start_prepare_dinner.iloc[0])
            chef_durring_dinner = roundOf(chef_durring_dinner)
        chef.at[0, reciption]= chef_durring_dinner  

        kitchen_assistants_during_breakfast=(breakfast_target_time.iloc[0]+breakfast_target_buffet_time.iloc[0])/(breakfast_start.iloc[0]-breakfast_prepare_start.iloc[0])

        dining_1stshift_start = dining_start.iloc[0]
        dining_1stshift_end = 0.0
        dining_2ndtshift_start = 0.0
        dining_2ndtshift_end = 0.0
        if dining_split_shifts.iloc[0] == "no":
            dining_1stshift_end = dining_end.iloc[0]
        else:
            dining_1stshift_end = (dining_end.iloc[0]-dining_start.iloc[0])/2+dining_start.iloc[0]-time_between_splits.iloc[0]/2

        if dining_split_shifts.iloc[0]  == "yes":
            dining_2ndtshift_start = (dining_end.iloc[0]-dining_start.iloc[0])/2+dining_start.iloc[0]+time_between_splits.iloc[0]/2
            dining_2ndtshift_end = dining_end.iloc[0]

        dining_shifts_df.at[0,reciption]= dining_1stshift_start  
        dining_shifts_df.at[1,reciption]= dining_1stshift_end  
        dining_shifts_df.at[2,reciption]= dining_2ndtshift_start
        dining_shifts_df.at[3,reciption]= dining_2ndtshift_end
        kitchen_during_diner=kitchen_assistants_during_diner_cal(menue_type, culinairy_complex_total, culinairy_simple_total, boardtype2, target_kitchen_cleaning_time, dining_split_shifts, dining_1stshift_end, start_prepare_dinner, dining_2ndtshift_end, dining_2ndtshift_start,chef_type , cooking_simple_total, cooking_complex_total)

        if type(kitchen_assistants_during_breakfast)!= np.float64:
            kitchen_assistants.at[0, reciption]= roundOf(kitchen_assistants_during_breakfast.iloc[0])
        else:
            kitchen_assistants.at[0, reciption]= roundOf(kitchen_assistants_during_breakfast)
        if type(kitchen_during_diner)!= np.float64:
            kitchen_assistants.at[1,reciption] = roundOf(kitchen_during_diner.iloc[0])
        else:
            kitchen_assistants.at[1,reciption] = roundOf(kitchen_during_diner)

        waiters_during_breakfast=((breakfast_target_time.iloc[0]+breakfast_target_buffet_time.iloc[0])+table_setting_total.iloc[0]* (boardtype1.iloc[0]+boardtype2.iloc[0]))/(breakfast_end.iloc[0]-breakfast_start.iloc[0])
        waiters_durring_dinner=waiters_during_diner_cal(mobile_ordering_paying,waitering_normal,waitering_mobile, dining_split_shifts, table_setting_total, boardtype2, occupancy_rates, guests_take_premade, premade_meals_option, dining_1stshift_end, dining_1stshift_start, dining_2ndtshift_end, dining_2ndtshift_start)
        waiters.at[0,reciption] = roundOf(waiters_during_breakfast)
        if type(waiters_during_breakfast)!=np.float64:
            waiters.at[0,reciption] = roundOf(waiters_during_breakfast.iloc[0])
        else:
            waiters.at[0,reciption] = roundOf(waiters_during_breakfast) 
        if type(waiters_durring_dinner)!=np.float64:
            waiters.at[1,reciption] = roundOf(waiters_durring_dinner.iloc[0])
        else:
            waiters.at[1,reciption] = roundOf(waiters_durring_dinner)


        bartenders_during_bartime=(24.0-bar_open.iloc[0])/target_average_workhours
        bartenders.at[0,reciption] = roundOf(bartenders_during_bartime)

        spa_therapists_during_massage_time=massage_reservations.iloc[0]*spa_massage_duration.iloc[0]/(wellness_closed.iloc[0]-wellness_open.iloc[0])
        spa_wellness.at[0,reciption]=  spa_therapists_during_massage_time
        

    
    peak_demand_dict={"head_of_reception":head_of_reception, "head_of_cleaning":head_of_cleaning, "receptionists":receptionists, "head_of_restaurant":head_of_restaurant, "room_cleaners": room_cleaners, "common_area_clean":common_area_clean, "laundry_attends":laundry_attends, "chef":chef, "kitchen_assistants":kitchen_assistants,"waiters":waiters, "bartenders":bartenders, "spa_wellness":spa_wellness,"labor_demand_df":labor_demand_df,"peak_demand_df": peak_demand_df, "bar_df": bar_df, "meal_times_df": meal_times_df, "spa_wellness_df": spa_wellness_df, "dining_shifts_df":dining_shifts_df }
    dataframes_df = list(peak_demand_dict.values())

 
    excel_writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')

    # Start row to keep track of current row position
    start_row = 0

    # Loop through the DataFrames and write each one to the Excel file
    for idx, df in enumerate(dataframes_df):
        if idx > 0:
            # Insert an empty row
            start_row += 1
        df.to_excel(excel_writer, sheet_name='Sheet1', startrow=start_row, index=False)
        start_row += df.shape[0] + 1  # Move to the next row after the dataframe
        
    excel_writer.close()
    return peak_demand_dict


In [19]:
peak_demand_df_list = peak_demand(dataframes)

In [ ]:
pip install xlsxwriter

In [20]:
def receptionists_formula(start_time, end_time, check_in_start, check_in_end, check_out_start, check_out_end, reciption_check_in, reciption_check_out, reciption_check_regular):
    if end_time==0.0:
        end_time=24.0
    if start_time >= check_in_start and end_time <= check_in_end:
       
        return reciption_check_in
    elif start_time >= check_out_start and end_time <= check_out_end:
        
        return reciption_check_out
    else:
        return reciption_check_regular

def room_cleaner_formula(start_time, end_time, check_in_earliest, check_out_latest, room_clean_start, room_clean_between_checks, room_clean_regular):  
    if end_time==0.0:
        end_time=24.0
    
    if check_out_latest <= start_time <= check_in_earliest and check_out_latest <= end_time <= check_in_earliest:
        return room_clean_between_checks
    elif end_time > room_clean_start and end_time <= room_clean_start + 8/24:
        return room_clean_regular
    else:
        return ""
    
def chef_formula(start_time,end_time, start_preperation_diner, dining_end, chef_during_diner):
    #solve
    if end_time==0.0:
        end_time=24.0  
    if start_preperation_diner <= start_time <= dining_end and start_preperation_diner <= end_time <= dining_end:
        return chef_during_diner
    else:
        return ""
def kitchen_assistants_formula(start_time, end_time, breakfast_preperation , breakfast_start, start_preperation_diner, dining_end, Kitchen_assistants_during_breakfast,Kitchen_assistants_during_diner):  
    #not same
    if end_time==0.0:
        end_time=24.0
    if (start_time >= breakfast_preperation) and (end_time <= breakfast_start):
        return Kitchen_assistants_during_breakfast
    elif (start_time >= start_preperation_diner) and (end_time <= dining_end):
        return Kitchen_assistants_during_diner
    else:
        return ""
    
def waiters_formula(start_time, end_time, breakfast_start, breakfast_end, dining_split_shifts, dining_start,dining_end, dining_1stshift_end,dining_2ndtshift_start, dining_2ndtshift_end, waiters_during_breakfast, waiters_during_diner):  
    if end_time==0.0:
        end_time=24.0
    if (start_time >= breakfast_start) and (end_time <= breakfast_end):
        return waiters_during_breakfast
    elif dining_split_shifts == "no":
        if (start_time >= dining_start) and (end_time <= dining_end):
            return waiters_during_diner
        else:
            return ""
    elif ( (start_time >= dining_start) and (end_time <= dining_1stshift_end) ) or ( (start_time >= dining_2ndtshift_start) and (end_time <= dining_2ndtshift_end) ):
        return waiters_during_diner
    else:
        return ""
    
def bartenders_formula(start_time, end_time, bar_open, bar_closed, bartenders_during_bartime):
    #not same
    if end_time==0.0:
        end_time=24.0
    if start_time >= bar_open and end_time <= bar_closed:
        return bartenders_during_bartime
    else:
        return ""
    
def spa_therapists_formula(start_time, end_time, wellness_open, wellness_closed, spa_therapists_during_massage_time):
    if end_time==0.0:
        end_time=24.0   
    if (start_time >= wellness_open) and (end_time <= wellness_closed):
        return spa_therapists_during_massage_time
    else:
        return ""



In [21]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

def intraday_agenda_rounded(dataframes, peak_demand_df_list):
    peak_demand_df = peak_demand_df_list["peak_demand_df"]
    meal_times_df = peak_demand_df_list["meal_times_df"]
    bar_df = peak_demand_df_list["bar_df"]
    spa_wellness_df = peak_demand_df_list["spa_wellness_df"]
    head_of_reception = peak_demand_df_list["head_of_reception"]
    head_of_cleaning = peak_demand_df_list["head_of_cleaning"]
    receptionists = peak_demand_df_list["receptionists"]
    head_of_restaurant = peak_demand_df_list["head_of_restaurant"]
    room_cleaners = peak_demand_df_list["room_cleaners"]
    chef = peak_demand_df_list["chef"]
    kitchen_assistants = peak_demand_df_list["kitchen_assistants"]
    waiters =peak_demand_df_list["waiters"]
    bartenders = peak_demand_df_list["bartenders"]
    spa_wellness = peak_demand_df_list["spa_wellness"]
    dining_shifts_df = peak_demand_df_list["dining_shifts_df"]
    room_clean_time_start=add_date_columns(dataframes["room_cleaning"])
    if len(room_clean_time_start.values)==0:
        room_clean_time_start=add_date_columns(dataframes["room_cleaning"])

    # Define the start and end times
    start_time = datetime.strptime("00:00", "%H:%M")
    end_time = datetime.strptime("23:45", "%H:%M")
    time_difference = timedelta(minutes=15)

    data = []

    # Generate the time range with a 15-minute difference
    current_time = start_time
    while current_time <= end_time:
        data.append({
            "Starttime": current_time.strftime("%H:%M"),
            "Endtime": (current_time + time_difference).strftime("%H:%M")
        })
        current_time += time_difference

    # Create a pandas DataFrame
    intraday_agenda_dict = {}

    # Iterate through the columns in peak_demand DataFrame
    for date in head_of_reception.columns[1:]:
        intraday_agenda_dict[date] = pd.DataFrame(data)

    combined_df = pd.DataFrame()
    result_dic={}
    for date, df in intraday_agenda_dict.items():
        reception_job = []
        reception_amount = []
        room_celaner_job = []
        room_celaner_amount = []
        chef_job = []
        chef_amount = []
        kitchen_job = []
        kitchen_amount = []
        waiters_job = []
        waiters_amount = []
        bartenders_job = []
        bartenders_amount = []
        spa_job = []
        spa_amount = []
        for index, row in intraday_agenda_dict[date].iterrows():
            start_time =row["Starttime"]
            end_time = row["Endtime"]

            start_time_obj = datetime.strptime(start_time, "%H:%M").time()
            start_time = convert_to_hours(start_time_obj)
            end_time_obj = datetime.strptime(end_time, "%H:%M").time()
            end_time = convert_to_hours(end_time_obj)

            check_in_start=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-in peak start"][date].apply(convert_to_hours)
            check_in_end=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-in peak end"][date].apply(convert_to_hours)
            check_out_start=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-out peak start"][date].apply(convert_to_hours)
            check_out_end=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-out peak end"][date].apply(convert_to_hours)

            check_in_earliest=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-in time (earliest)"][date].apply(convert_to_hours)
            check_out_latest=peak_demand_df[peak_demand_df["Check-in and out times"]=="check-out time (latest)"][date].apply(convert_to_hours)
            room_clean_start=room_clean_time_start[room_clean_time_start["Room cleaning time start"]=="Room cleaning time start"][date].apply(convert_to_hours)

            start_prepare_dinner=meal_times_df[meal_times_df["Meal times"]=="start preperation diner"][date].apply(convert_to_hours)
            dining_start=meal_times_df[meal_times_df["Meal times"]=="dining start"][date].apply(convert_to_hours)
            dining_end=meal_times_df[meal_times_df["Meal times"]=="dining end"][date].apply(convert_to_hours)
            breakfast_preperation=meal_times_df[meal_times_df["Meal times"]=="breakfast preperation"][date].apply(convert_to_hours)
            breakfast_start=meal_times_df[meal_times_df["Meal times"]=="breakfast start"][date].apply(convert_to_hours)
            breakfast_end=meal_times_df[meal_times_df["Meal times"]=="breakfast end"][date].apply(convert_to_hours)
            dining_split_shifts=meal_times_df[meal_times_df["Meal times"]=="dining split shifts"][date]

            dining_1stshift_end=dining_shifts_df[dining_shifts_df["Dining Shifts"]=="dining_1stshift_end"][date]
            dining_2ndshift_start=dining_shifts_df[dining_shifts_df["Dining Shifts"]=="dining_2ndshift_start"][date]
            dining_2ndshift_end = dining_shifts_df[dining_shifts_df["Dining Shifts"]=="dining_2ndshift_end"][date]

            bar_open = bar_df[bar_df["Bar"]=="Bar open"][date].apply(convert_to_hours)
            bar_closed = 24.0

            wellness_open = spa_wellness_df[spa_wellness_df["Spa and Wellness"]=="Wellness open"][date].apply(convert_to_hours)
            wellness_closed = spa_wellness_df[spa_wellness_df["Spa and Wellness"]=="Wellness closed"][date].apply(convert_to_hours)
            receptionists_formula_result=receptionists_formula(start_time, end_time, check_in_start.iloc[0], check_in_end.iloc[0], check_out_start.iloc[0], check_out_end.iloc[0], receptionists.iloc[0][0], receptionists.iloc[1][0], receptionists.iloc[2][0])

            try:
                if receptionists_formula_result in receptionists["Receptionists"].values:
                    reception_job.append(receptionists_formula_result)
                    amount=receptionists[receptionists["Receptionists"]== receptionists_formula_result][date]
                    reception_amount.append(np.ceil(amount.iloc[0]))                
            except:
                pass

            room_cleaner_formula_result=room_cleaner_formula(start_time, end_time, check_in_earliest.iloc[0], check_out_latest.iloc[0], room_clean_start.iloc[0], room_cleaners.iloc[0][0], room_cleaners.iloc[1][0])
            try:
                if room_cleaner_formula_result in room_cleaners["Room cleaners"].values:
                    room_celaner_job.append(room_cleaner_formula_result)
                    value=room_cleaners[room_cleaners["Room cleaners"]=="Room cleaner regular"][date]
                    if room_cleaner_formula_result == "Room cleaner between checks":
                        amount=room_cleaners[room_cleaners["Room cleaners"]== room_cleaner_formula_result][date] 
                        amount = amount.iloc[0] + value.iloc[0]
                    else:
                        amount=room_cleaners[room_cleaners["Room cleaners"]== room_cleaner_formula_result][date]
                        amount=amount.iloc[0]
                    room_celaner_amount.append(np.ceil(amount))
                else:
                    room_celaner_job.append(room_cleaner_formula_result)
                    room_celaner_amount.append(0)
            except:
                pass
            
            chef_formula_result=chef_formula(start_time,end_time, start_prepare_dinner.iloc[0], dining_end.iloc[0], chef.iloc[0][0])
            try:
                if chef_formula_result in chef["Chef"].values:
                    chef_job.append(chef_formula_result)
                    amount=chef[chef["Chef"]== chef_formula_result][date]
                    chef_amount.append(np.ceil(amount.iloc[0]))
                else:
                    chef_job.append(chef_formula_result)
                    chef_amount.append(0)

            except:
                pass

            kitchen_assistants_formula_result=kitchen_assistants_formula(start_time,end_time, breakfast_preperation.iloc[0] , breakfast_start.iloc[0], start_prepare_dinner.iloc[0], dining_end.iloc[0], kitchen_assistants.iloc[0][0],kitchen_assistants.iloc[1][0])

            try:
                if kitchen_assistants_formula_result in kitchen_assistants["Kitchen assistants"].values:
                    kitchen_job.append(kitchen_assistants_formula_result)
                    amount=kitchen_assistants[kitchen_assistants["Kitchen assistants"]== kitchen_assistants_formula_result][date]
                    kitchen_amount.append(np.ceil(amount.iloc[0]))
                else:
                    kitchen_job.append(kitchen_assistants_formula_result)
                    kitchen_amount.append(0)

            except:
                pass

            waiters_formula_result=waiters_formula(start_time, end_time, breakfast_start.iloc[0], breakfast_end.iloc[0], dining_split_shifts.iloc[0], dining_start.iloc[0] ,dining_end.iloc[0] , dining_1stshift_end.iloc[0] ,dining_2ndshift_start.iloc[0] , dining_2ndshift_end.iloc[0] , waiters.iloc[0][0],  waiters.iloc[1][0])

            try:
                if waiters_formula_result in waiters["Waiters"].values:
                    waiters_job.append(waiters_formula_result)
                    amount=waiters[waiters["Waiters"]== waiters_formula_result][date]
                    waiters_amount.append(np.ceil(amount.iloc[0]))
                else:
                    waiters_job.append(waiters_formula_result)
                    waiters_amount.append(0)

            except:
                pass

            bartenders_formula_result=bartenders_formula(start_time, end_time, bar_open.iloc[0], bar_closed, bartenders.iloc[0][0])

            try:
                if bartenders_formula_result in bartenders["Bartenders"].values:
                    bartenders_job.append(bartenders_formula_result)
                    amount=bartenders[bartenders["Bartenders"]== bartenders_formula_result][date]
                    bartenders_amount.append(np.ceil(amount.iloc[0]))
                else:
                    bartenders_job.append(bartenders_formula_result)
                    bartenders_amount.append(0)

            except:
                pass
            spa_therapists_formula_result=spa_therapists_formula(start_time, end_time, wellness_open.iloc[0], wellness_closed.iloc[0], spa_wellness.iloc[0][0])

            try:
                if spa_therapists_formula_result in spa_wellness["Spa therapist"].values:
                    spa_job.append(spa_therapists_formula_result)
                    amount=spa_wellness[spa_wellness["Spa therapist"]== spa_therapists_formula_result][date]
                    spa_amount.append(np.ceil(amount.iloc[0]))
                else:
                    spa_job.append(spa_therapists_formula_result)
                    spa_amount.append(0)

            except:
                pass
        intraday_agenda_dict[date][f"reception_job"]=reception_job
        intraday_agenda_dict[date][f"reception_amount"]=reception_amount
        intraday_agenda_dict[date][f"room_celaner_job"]=room_celaner_job
        intraday_agenda_dict[date][f"room_celaner_amount"]=room_celaner_amount
        intraday_agenda_dict[date][f"chef_job"]=chef_job
        intraday_agenda_dict[date][f"chef_amount"]= chef_amount
        intraday_agenda_dict[date][f"kitchen_job"]= kitchen_job
        intraday_agenda_dict[date][f"kitchen_amount"]=kitchen_amount
        intraday_agenda_dict[date][f"waiters_job"]=waiters_job
        intraday_agenda_dict[date][f"waiters_amount"]=waiters_amount
        intraday_agenda_dict[date][f"bartenders_job"]=bartenders_job
        intraday_agenda_dict[date][f"bartenders_amount"]=bartenders_amount
        intraday_agenda_dict[date][f"spa_job"]=spa_job
        intraday_agenda_dict[date][f"spa_amount"]=spa_amount
        time_duration = timedelta(hours=0, minutes=15, seconds=0)
        column_sums = time_duration * intraday_agenda_dict[date].sum(numeric_only=True)
        column_total = column_sums.apply(lambda x: timedelta(days=x.days))
        total_hours = column_sums.dt.days * 24 + column_sums.dt.seconds // 3600
        total_minutes = (column_sums.dt.seconds // 60) % 60
        time_format = [f"{hours:02d}:{minutes:02d}" for hours, minutes in zip(total_hours, total_minutes)]
        time_format_series=pd.Series(time_format, index=total_minutes.index)

        # Append totals as a new row
        result_dic[date]=time_format_series
        intraday_agenda_dict[date].loc["Total"] = time_format_series
        combined_df = pd.concat([combined_df, df], axis=1)
        combined_df.to_excel("Intraday_Agenda(Rounded).xlsx", index=False, engine='openpyxl')
    return result_dic

In [22]:
intraday_agenda_df_list= intraday_agenda_rounded(dataframes, peak_demand_df_list)

In [23]:
def add_time_strings(time_str1, time_str2):
    hours1, minutes1 = map(int, time_str1.split(':'))
    hours2, minutes2 = map(int, time_str2.split(':'))
    
    total_hours = hours1 + hours2
    total_minutes = minutes1 + minutes2
    
    if total_minutes >= 60:
        total_hours += total_minutes // 60
        total_minutes %= 60
    
    result_time_str = f"{total_hours}:{total_minutes:02d}"
    return result_time_str

In [24]:
def format_timedelta(td):
    total_minutes = td.total_seconds() // 60
    hours = int(total_minutes // 60)
    minutes = int(total_minutes % 60)
    return f"{hours:02d}:{minutes:02d}"

In [27]:
def employee_model_final_output(peak_demand_df_list, intraday_agenda_df_list):
    result_dic = intraday_agenda_df_list
    try:
        labor_demand_df=peak_demand_df_list["labor_demand_df"]
        estimation_output= pd.DataFrame(columns=["Week","KPI's (weekly averages}","Occupancy rate" ,"New guests","Leaving guests","BB guests","HB guests","FB guests"])
        occupancy_rates= labor_demand_df[labor_demand_df["Reservations"] == "Occupancy rate"]
        new_guest= labor_demand_df[labor_demand_df["Reservations"] == "New guests"]
        leaving_guests= labor_demand_df[labor_demand_df["Reservations"] == "Leaving guests"]
        boardtype1= labor_demand_df[labor_demand_df["Reservations"] == "Boardtype1"]
        boardtype2= labor_demand_df[labor_demand_df["Reservations"] == "Boardtype2"]
        boardtype3= labor_demand_df[labor_demand_df["Reservations"] == "Boardtype3"]
        avarage_values=occupancy_rates[list(result_dic.keys())].mean()
        estimation_output["Occupancy rate"]= [roundOf(avarage_values.mean())]
        avarage_values=new_guest[list(result_dic.keys())].mean()
        estimation_output["New guests"]= avarage_values.mean()
        avarage_values=leaving_guests[list(result_dic.keys())].mean()
        estimation_output["Leaving guests"]= avarage_values.mean()
        avarage_values=boardtype1[list(result_dic.keys())].mean()
        estimation_output["BB guests"]= int(avarage_values.mean())
        avarage_values=boardtype2[list(result_dic.keys())].mean()
        estimation_output["HB guests"]= int(avarage_values.mean())
        avarage_values=boardtype3[list(result_dic.keys())].mean()
        estimation_output["FB guests"]= int(avarage_values.mean())
        tod= datetime.today()
        week_number = tod.isocalendar()[1]
        estimation_output["Week"]=week_number

        employee_output= pd.DataFrame()
        employee_output["Date"]=["Department","Cleaning","Kitchen","Waitering","Reception","Other", "Total"]    
        employee_output=add_date_columns(employee_output)
       
        agenda_df = pd.DataFrame(columns=result_dic.keys())
        
        result_df = pd.concat([employee_output, agenda_df], axis=1)
    except Exception as exception:
        raise exception
        
    cleaning=[]
    kitchen=[]
    waitering=[]
    reception =[]
    others = []
    if len(result_dic.keys())>0:
        for k in result_dic:
            reception.append(result_dic[k][0])
            cleaning.append(result_dic[k][1])
            waitering.append(result_dic[k][4])
            kitchen_total=add_time_strings(result_dic[k][2],result_dic[k][3])
            kitchen.append(kitchen_total)
            others_total= add_time_strings(result_dic[k][5],result_dic[k][6])
            others.append(others_total)
        colum_ind = result_df.columns[1]    
        reception_index=result_df[result_df["Department"]=="Reception"].index[0]-1
        clean_index=result_df[result_df["Department"]=="Cleaning"].index[0]-1
        kitchen_index=result_df[result_df["Department"]=="Kitchen"].index[0]-1
        waiter_index=result_df[result_df["Department"]=="Waitering"].index[0]-1
        other_index=result_df[result_df["Department"]=="Other"].index[0]-1
        second_column_index = result_df.columns.get_loc(colum_ind)
        result_df.iloc[clean_index,second_column_index:]=cleaning
        result_df.iloc[reception_index,second_column_index:]=reception
        result_df.iloc[kitchen_index,second_column_index:]=kitchen
        result_df.iloc[waiter_index,second_column_index:]=waitering
        result_df.iloc[other_index,second_column_index:]=others
        
        sum_per_date = {}
        for date_col in result_df.columns[1:]:
            total_time_per_date = timedelta(seconds=0)
            for index, row in result_df.iterrows():
                value = row[date_col]
                if isinstance(value, str):
                    hours, minutes = map(int, value.split(":"))
                    total_time_per_date += timedelta(hours=hours, minutes=minutes)
            sum_per_date[date_col] = total_time_per_date

        # Convert the sum of time to a list and update the total row
        sum_list = ["Total"]
        for date_col in result_df.columns[1:]:
            total_time = sum_per_date[date_col]
            total_hours = total_time.days * 24 + total_time.seconds // 3600
            total_minutes = (total_time.seconds // 60) % 60
            total_str = f"{total_hours}:{total_minutes:02d}"
            sum_list.append(total_str)
  
        # Append the sum of time as a new row and update the DataFrame
        result_df.loc[len(result_df)] = sum_list 
 

        # Display the resulting DataFrame
        dataframes_df = [estimation_output, result_df]

 
        excel_writer = pd.ExcelWriter('final_output.xlsx', engine='xlsxwriter')

        # Start row to keep track of current row position
        start_row = 0

        # Loop through the DataFrames and write each one to the Excel file
        for idx, df in enumerate(dataframes_df):
            if idx > 0:
                # Insert an empty row
                start_row += 1
            df.to_excel(excel_writer, sheet_name='Sheet1', startrow=start_row, index=False)
            start_row += df.shape[0] + 1  # Move to the next row after the dataframe

        excel_writer.close()
        return print("Successully dumped employee model result in final_output.xlsx file")
    else:
        print("No data in infraday agenday dataframes")

In [28]:
employee_model_final_output(peak_demand_df_list, intraday_agenda_df_list)

Successully dumped employee model result in final_output.xlsx file


In [ ]:
peak_demand_df_list["spa_wellness"]